In [2]:
# necessary imports
%matplotlib inline
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from IPython.display import display_html
import bs4 as bs
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from pandas import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import *
import lxml
import html5lib
import openpyxl
import xlsxwriter
import time
import os.path

In [3]:
def html_table2df(soup):
    t = str(soup.find('div', class_='nba-stat-table__overflow')['table'])
    
    df = pd.read_html(t)[0]
    df.rename(columns={'Unnamed: 0': 'rank'}, inplace=True)
    df = df.iloc[:, :df.columns.get_loc('PIE')+1]
    return df

def store(dataframe, sheet_name, file_name):
    writer = pd.ExcelWriter(file_name +'.xlsx', engine='xlsxwriter')
    for i in ['sh2', 'sh3', 'sh4']:
        sheet_name = i
        dataframe.to_excel(writer, sheet_name=sheet_name)
    
    writer.save()


def store_excel(dataframe, sheet_name, file_name):
    dataframe.rename(columns={'Unnamed: 0': 'rank'}, inplace=True)
    dataframe = df.iloc[:, :df.columns.get_loc('PIE')+1]
    
    if not os.path.isfile(file_name):
        #create new and save it    檔案存在 會覆蓋
        workbook_name = file_name
        wb = Workbook()
        wb.save(workbook_name)  
    else:
        wb = load_workbook(file_name)
        wb.create_sheet(sheet_name)
        ws = wb.get_sheet_by_name(sheet_name)
        for r in dataframe_to_rows(df, index=False, header=True):
            ws.append(r)
        wb.save(file_name)



In [4]:
# 產生資料
season_lst = [str(2017-i) + '-'  for i in range(20)]
lst = list()
for i in season_lst:
    pre_str = i
    num = int(i[2:4]) + 1
    if num < 10:
        lst.append(pre_str + str(0) + str(num))
    else:
        lst.append(pre_str + str(num))
    
print(lst)

['2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13', '2011-12', '2010-11', '2009-10', '2008-09', '2007-08', '2006-07', '2005-06', '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-100', '1998-99']


In [5]:
browser = webdriver.Chrome('./chromedriver')

In [6]:
# params
# Season: 2017-18
# SeasonType: Regular%20Season

season = '2016-17'
season_type = 'Regular%20Season'

my_url = 'https://stats.nba.com/teams/advanced/?sort=W&dir=-1&Season={}&SeasonType={}'.format(season, season_type)
browser.get(my_url)

In [7]:
page_source = browser.page_source

In [8]:
soup = bs.BeautifulStoneSoup(page_source)

c:\users\linus\pycharmprojects\ccproject_nbada\lib\site-packages\bs4\__init__.py:568: UserWarning: The BeautifulStoneSoup class is deprecated. Instead of using it, pass features="xml" into the BeautifulSoup constructor.
  'The BeautifulStoneSoup class is deprecated. Instead of using '


In [9]:
t = soup.find('div', class_='nba-stat-table__overflow').table
t = str(t)
df = pd.read_html(t)[0]
df.head()

,Unnamed: 0,TEAM,GP,W,L,MIN,OffRtg,DefRtg,NetRtg,AST%,...,AST/TO RANK,ASTRatio RANK,OREB% RANK,DREB% RANK,REB% RANK,TORatio RANK,eFG% RANK,TS% RANK,PACE RANK,PIE RANK
0,1,Golden State Warriors,82,67,15,3956.0,114.8,103.4,11.3,70.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,San Antonio Spurs,82,61,21,3961.0,110.3,102.8,7.5,60.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Houston Rockets,82,55,27,3956.0,114.1,108.4,5.6,62.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Boston Celtics,82,53,29,3951.0,110.6,108.0,2.6,65.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Cleveland Cavaliers,82,51,31,3976.0,112.8,109.7,3.0,56.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
wb = load_workbook('nba_regu.xlsx')
wb.create_sheet(season)

ws = wb.get_sheet_by_name(season)
for r in dataframe_to_rows(df, index=True, header=True):
    ws.append(r)

wb.save('nba_regu.xlsx')

c:\users\linus\pycharmprojects\ccproject_nbada\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  after removing the cwd from sys.path.


In [12]:
# try one flow
#season = '2016-17'
season_type = 'Playoffs'
l = ['2017-18', '2016-17', '2015-16', '2014-15', '2013-14']
for season in l:
    browser = webdriver.Chrome('./chromedriver')
    my_url = 'https://stats.nba.com/teams/advanced/?sort=W&dir=-1&Season={}&SeasonType={}'.format(season, season_type)
    browser.get(my_url)
    page_source = browser.page_source
    browser.close()
    soup = bs.BeautifulStoneSoup(page_source)
    t = soup.find('div', class_='nba-stat-table__overflow').table
    t = str(t)
    df = pd.read_html(t)[0]
    df.head()


    df.rename(columns={'Unnamed: 0': 'rank'}, inplace=True)
    df = df.iloc[:, :df.columns.get_loc('PIE')+1]
    df.head()

    wb = load_workbook('nba_playoff_five_year.xlsx')
    wb.create_sheet(season)

    ws = wb.get_sheet_by_name(season)
    for r in dataframe_to_rows(df, index=True, header=True):
        ws.append(r)

    wb.save('nba_playoff_five_year.xlsx')


FileNotFoundError: [Errno 2] No such file or directory: 'nba_playoff_five_year.xlsx'

In [ ]:
workbook_name = 'nba_playoff_five_year.xlsx'
wb = Workbook()
wb.save(workbook_name)

In [ ]:
# create new and save it
workbook_name = 'nba_regu.xlsx'
wb = Workbook()
wb.save(workbook_name)

url = 'https://stats.nba.com/teams/advanced/?sort=W&dir=-1&Season={}&SeasonType={}'

url = url.format('2017-18', 'Regular%20Season')

    
# writer = pd.ExcelWriter('nba_regu' +'.xlsx', engine='xlsxwriter')
# df.to_excel(writer, sheet_name=season)
# time.sleep(0.1)
        
# writer.save()    


In [ ]:
df.set_index(drop=True,inplace=True)

In [ ]:
wb = load_workbook(workbook_name)
wb.create_sheet('shet3')

ws = wb.get_sheet_by_name("shet3")
for r in dataframe_to_rows(df, index=True, header=True):
    ws.append(r)

wb.save(workbook_name)


In [37]:
# store
store(df, 'sh2','t2')
store(df, 'sh3','t2')
store(df, 'sh4','t2')


In [13]:
from IPython.display import display_html
display_html(t, raw=True)

TEAM 
 GP 
 W 
 L 
 MIN 
 OffRtg 
 DefRtg 
 NetRtg 
 AST% 
 AST/TO 
 AST Ratio 
 OREB% 
 DREB% 
 REB% 
 TOV% 
 eFG% 
 TS% 
 PACE 
 PIE 
 GP RANK 
 W RANK 
 L RANK 
 MIN RANK 
 OffRtg RANK 
 DefRtg RANK 
 NetRtg RANK 
 AST% RANK 
 AST/TO RANK 
 AST Ratio RANK 
 OREB% RANK 
 DREB% RANK 
 REB% RANK 
 TO Ratio RANK 
 eFG% RANK 
 TS% RANK 
 PACE RANK 
 PIE RANK 
 
 
 
<!-- --> 
 1 
 Golden State Warriors 
 21 
 16 
 5 
 1013.0 
 112.1 
 101.7 
 10.3 
 63.2 
 2.03 
 19.1 
 25.5 
 73.2 
 50.8 
 13.0 
 54.5 
 58.1 
 98.13 
 57.2 
 <!-- --> 
 2 
 Cleveland Cavaliers 
 22 
 12 
 10 
 1066.0 
 108.1 
 110.7 
 -2.5 
 52.2 
 1.52 
 15.3 
 28.9 
 74.4 
 51.3 
 13.6 
 51.2 
 55.0 
 92.38 
 48.3 
 <!-- --> 
 3 
 Boston Celtics 
 19 
 11 
 8 
 922.0 
 107.0 
 106.0 
 1.0 
 58.2 
 1.82 
 16.5 
 25.6 
 74.7 
 49.4 
 12.3 
 50.0 
 54.1 
 93.81 
 49.9 
 <!-- --> 
 3 
 Houston Rockets 
 17 
 11 
 6 
 816.0 
 108.3 
 106.4 
 1.8 
 50.6 
 1.56 
 14.7 
 25.1 
 75.5 
 49.4 
 12.4 
 51.4 
 55.2 
 96.91 
 49.7 
 <!-- --> 
 5 
 New Orleans Pelicans 
 9 
 5 
 4 
 432.0 
 105.4 
 106.7 
 -1.3 
 62.7 
 1.86 
 19.0 
 21.5 
 76.2 
 49.7 
 13.8 
 53.3 
 56.2 
 104.06 
 50.4 
 <!-- --> 
 5 
 Philadelphia 76ers 
 10 
 5 
 5 
 485.0 
 108.9 
 104.9 
 4.0 
 66.7 
 1.77 
 18.0 
 33.3 
 76.9 
 54.3 
 14.6 
 49.3 
 53.9 
 98.92 
 53.2 
 <!-- --> 
 5 
 Utah Jazz 
 11 
 5 
 6 
 528.0 
 104.0 
 106.1 
 -2.1 
 53.7 
 1.30 
 15.5 
 25.5 
 75.4 
 50.7 
 15.8 
 52.0 
 54.9 
 98.50 
 48.8 
 <!-- --> 
 8 
 Toronto Raptors 
 10 
 4 
 6 
 485.0 
 111.3 
 116.1 
 -4.8 
 56.0 
 1.56 
 16.7 
 27.2 
 71.9 
 49.6 
 14.5 
 53.6 
 57.8 
 95.31 
 46.9 
 <!-- --> 
 9 
 Indiana Pacers 
 7 
 3 
 4 
 336.0 
 109.8 
 103.4 
 6.4 
 54.8 
 1.61 
 17.0 
 27.4 
 73.5 
 50.7 
 14.4 
 54.2 
 56.7 
 91.64 
 54.5 
 <!-- --> 
 9 
 Milwaukee Bucks 
 7 
 3 
 4 
 341.0 
 108.5 
 108.8 
 -0.3 
 56.2 
 1.67 
 17.8 
 19.0 
 69.8 
 46.6 
 14.0 
 56.1 
 58.5 
 92.62 
 50.9 
 <!-- --> 
 11 
 Oklahoma City Thunder 
 6 
 2 
 4 
 288.0 
 102.9 
 106.9 
 -4.0 
 43.9 
 1.17 
 12.8 
 26.1 
 71.3 
 48.5 
 14.2 
 49.0 
 52.8 
 98.50 
 46.9 
 <!-- --> 
 11 
 Washington Wizards 
 6 
 2 
 4 
 288.0 
 109.4 
 110.8 
 -1.5 
 58.8 
 1.73 
 17.4 
 27.9 
 73.6 
 50.4 
 13.8 
 51.3 
 56.1 
 98.25 
 49.8 
 <!-- --> 
 13 
 Miami Heat 
 5 
 1 
 4 
 240.0 
 101.8 
 112.2 
 -10.4 
 65.0 
 1.49 
 17.3 
 23.5 
 65.5 
 44.9 
 15.7 
 50.7 
 54.2 
 101.70 
 42.9 
 <!-- --> 
 13 
 Minnesota Timberwolves 
 5 
 1 
 4 
 240.0 
 106.9 
 115.7 
 -8.8 
 58.9 
 1.78 
 17.4 
 24.5 
 74.5 
 50.6 
 13.3 
 51.5 
 54.8 
 95.20 
 47.5 
 <!-- --> 
 13 
 San Antonio Spurs 
 5 
 1 
 4 
 240.0 
 102.1 
 111.6 
 -9.5 
 54.9 
 1.73 
 15.2 
 23.3 
 70.9 
 45.6 
 11.6 
 46.3 
 51.5 
 94.70 
 41.8 
 <!-- --> 
 16 
 Portland Trail Blazers 
 4 
 0 
 4 
 192.0 
 105.2 
 113.9 
 -8.7 
 46.1 
 1.31 
 14.3 
 26.7 
 78.1 
 49.9 
 14.5 
 51.2 
 53.9 
 100.38 
 42.6 
 <!-- -->